In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import matplotlib
import os
import scipy
from scipy.stats import shapiro, median_abs_deviation
import skimage as ski
from skimage.exposure import is_low_contrast

print("\nLibraries\n-----------------------------")
print(f"OpenCV: {cv2.__version__}")
print(f"Numpy: {np.__version__}")
print(f"Matplotlib: {matplotlib.__version__}")
print(f"SciPy: {scipy.__version__}")
print(f"SciKit Image: {ski.__version__}")

def true_value_exists(matrix):
    for row in matrix:
        if True in row:
            return True
    return False

def quantile_CI(x, q=[0.05, 0.95]):

    if not isinstance(x, (list, np.ndarray)) or len(x) == 0:

        raise ValueError("Quantile values can only be calculated on non-empty lists or arrays")

    if not isinstance(q, (list, np.ndarray)):

        raise ValueError("Quantiles must be provided as a list or array")

    if len(q) == 2:

        if not all(isinstance(quant, (int, float)) for quant in q):

            raise ValueError("Quantiles must be numeric")

        if any(quant < 0 or quant > 1 for quant in q):

            raise ValueError("Quantile values must be between 0 and 1")

        x_sorted = np.sort(x)
        lower_CI = x_sorted[int(q[0] * len(x_sorted))]
        upper_CI = x_sorted[int(q[1] * len(x_sorted))]

        return [lower_CI, upper_CI]

    elif len(q) == 1:

        if not isinstance(q[0], (int, float)):

            raise ValueError("Quantile must be numeric")

        x_sorted = np.sort(x)
        x_prima = x_sorted[int(q[0]) * len(x_sorted)]

        return x_prima

    else:

        raise ValueError("Invalid number of quantiles provided")


def descriptive_statistics(data, percentage = False):

    print(f"Shapiro results w = {shapiro(data)[0]:.2f}, p = {shapiro(data)[1]}")
    
    quantile_ranges = quantile_CI(data, [0.025, 0.975])

    if percentage:
        
        print(f"\nMin: {np.min(data):.2f} %")
        
        if shapiro(data)[1] < 0.003:
            
            print(f"Median: {np.median(data):.3f} %")
            print(f"NMAD: {(median_abs_deviation(data) * 1.4826):.3f}")
            
        else:
            
            print(f"Mean: {np.mean(data):.3f}%")
            print(f"Standard Deviation: {np.std(data):.3f}")

        print(
            f"95% Quantile Intevals: [{quantile_ranges[0]:.2f},",
            f"{quantile_ranges[1]:.2f}] %"
        )
        
        print(f"Max: {np.max(data):.2f} %")

    else:

        print(f"\nMin: {np.min(data):.2f}")

        if shapiro(data)[1] < 0.003:
            
            print(f"Median: {np.median(data):.3f}")
            print(f"NMAD: {(median_abs_deviation(data) * 1.4826):.3f}")
            
        else:
            
            print(f"Mean: {np.mean(data):.3f}")
            print(f"Standard Deviation: {np.std(data):.3f}")

        print(
            f"95% Quantile Intevals: [{quantile_ranges[0]:.2f},",
            f"{quantile_ranges[1]:.2f}] %"
        )
        print(f"Max: {np.max(data):.2f}")

class loadImage():

    def __init__(self, image):

        self.image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        low_contrast_bool = is_low_contrast(self.image, fraction_threshold = 0.35)
        if low_contrast_bool:
            self.adequate_contrast = False
        else:
            self.adequate_contrast = True

class LaplacianAnalysis(loadImage):

    def __init__(self, image):

        super().__init__(image)

        gaussian_blur = cv2.GaussianBlur(self.image, (5, 5), 0)
        self.laplacian_map = cv2.Laplacian(gaussian_blur, cv2.CV_64F, ksize = 3)
        self.laplacian_vector = self.laplacian_map.ravel()

        self.threshold = quantile_CI(np.abs(self.laplacian_vector), [0, 0.66]) [1]
    
    def calculate_variance(self):

        return(self.laplacian_vector.var())
    
    def calculate_quantile_range(self):

        interquantile_range = quantile_CI(
            self.laplacian_vector, [0.025, 0.975]
        )[1] - quantile_CI(
            self.laplacian_vector, [0.025, 0.975]
        )[0]

        return(interquantile_range)
    
    def binary_map(self):
        
        binary_matrix = (np.abs(self.laplacian_map) > self.threshold).astype(int)
        
        return(binary_matrix)
 

class CannyEdgeAnalysis(loadImage):

    def __init__(self, image):

        super().__init__(image)

        gaussian_blur = cv2.GaussianBlur(self.image, (5, 5), 0)
        self.edges = cv2.Canny(gaussian_blur, 10, 20)
    
    def calculate_features(self):

        dilate_kernel = np.ones((3, 3), np.uint8)
        dilated_edges = cv2.dilate(self.edges, dilate_kernel, iterations = 1)
        perc_detectable_features = (
            np.sum(dilated_edges.ravel() > 1) / dilated_edges.ravel().shape[0]
        ) * 100

        return(perc_detectable_features)

class FastFourierTransform(loadImage):

    def calculate_FFT(self, size = 60, robust = False):
        
        h, w = self.image.shape
        cX, cY = (int(w / 2), int(h / 2))
        
        fft = np.fft.fft2(self.image)
        fftShift = np.fft.fftshift(fft)
        
        fftShift[cY - size:cY + size, cX - size:cX + size] = 0
        fftShift = np.fft.ifftshift(fftShift)
        
        recon = np.fft.ifft2(fftShift)
        
        magnitude = 20 * np.log(np.abs(recon))
        magnitude = magnitude.ravel()

        if robust:
            
            shapiro_p_mag = shapiro(magnitude)[1]

            if shapiro_p_mag < 0.003:

                return(np.median(magnitude))
            
            else:

                return(np.mean(magnitude))

        else:
            
            return(np.mean(magnitude))

class SobelGradientMaps(loadImage):

    def __init__(self, image):

        super().__init__(image)

        self.sobel_x = cv2.Sobel(self.image, cv2.CV_64F, 1, 0)
        self.sobel_y = cv2.Sobel(self.image, cv2.CV_64F, 0, 1)

    def gradient_map(self):

        return(cv2.addWeighted(self.sobel_x, 0.5, self.sobel_y, 0.5, 0))
    
    def scaled_gradient_map(self):

        gX = cv2.convertScaleAbs(self.sobel_x)
        gY = cv2.convertScaleAbs(self.sobel_y)

        return(cv2.addWeighted(gX, 0.5, gY, 0.5, 0))
    
class SpectralReflection():

    def __init__(self, image, erode_iterations = 2):

        hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        B = np.mean(image[:,:,0].ravel())
        self.bin_image = hsv_image[:,:,2] > (B * 2)
        self.bin_image = self.bin_image.astype(np.uint8)
        
        (h, w) = image.shape[0], image.shape[1]
        
        kernel = np.ones((3, 3), np.uint8)
        self.clean_bin_image = cv2.erode(
            self.bin_image, kernel, iterations = erode_iterations
        )
        
        self.num_pixels = sum(self.bin_image.ravel())
        self.num_pixels_clean = sum(self.clean_bin_image.ravel())
        self.percentage = (self.num_pixels / (h * w)) * 100
        self.percentage_clean = (self.num_pixels_clean / (h * w)) * 100
        
        if self.num_pixels > 0:
            self.detected = True
        else:
            self.detected = False
            
        if self.num_pixels_clean > 0:
            self.detected_clean = True
        else:
            self.detected_clean = False


Libraries
-----------------------------
OpenCV: 4.7.0
Numpy: 1.22.0
Matplotlib: 3.7.0
SciPy: 1.7.3
SciKit Image: 0.19.3


In [2]:
main_directory = "./DS1"

for subfolder in os.listdir(main_directory):
    
    LoG_Variance = []
    LoG_Range = []
    
    subfolder_path = os.path.join(main_directory, subfolder)
    
    if os.path.isdir(subfolder_path):
        
        for file_name in os.listdir(subfolder_path):
            
            if file_name.lower().endswith((".bmp")):
                
                image_path = os.path.join(subfolder_path, file_name)
                image = cv2.imread(image_path)

                if image is not None:
                    
                    laplace = LaplacianAnalysis(image)
                    
                    LoGVar = laplace.calculate_variance()
                    LoGQR = laplace.calculate_quantile_range()
                    
                    LoG_Variance.append(LoGVar)
                    LoG_Range.append(LoGQR)
    
    print(f"\n{subfolder} -------------------------------------------------------\n")
    print("Statistics on LoG Variance\n")
    descriptive_statistics(LoG_Variance)

    print("\nStatistics on LoG Range\n")
    descriptive_statistics(LoG_Range)


Croc -------------------------------------------------------

Statistics on LoG Variance

Shapiro results w = 0.84, p = 1.8262286175740883e-05

Min: 16.97
Median: 71.010
NMAD: 46.242
95% Quantile Intevals: [17.78, 230.71] %
Max: 252.88

Statistics on LoG Range

Shapiro results w = 0.87, p = 0.00014010895392857492

Min: 16.00
Median: 30.000
NMAD: 8.896
95% Quantile Intevals: [16.00, 66.00] %
Max: 66.00

Cut Mark -------------------------------------------------------

Statistics on LoG Variance

Shapiro results w = 0.71, p = 3.5719387306436685e-28

Min: 11.90
Median: 20.660
NMAD: 6.002
95% Quantile Intevals: [13.85, 55.59] %
Max: 98.64

Statistics on LoG Range

Shapiro results w = 0.71, p = 2.5969708815541433e-28

Min: 12.00
Median: 16.000
NMAD: 0.000
95% Quantile Intevals: [14.00, 30.00] %
Max: 38.00


In [3]:
main_directory = "./DS3"

for subfolder in os.listdir(main_directory):
    
    LoG_Variance = []
    LoG_Range = []
    
    subfolder_path = os.path.join(main_directory, subfolder)
    
    if os.path.isdir(subfolder_path):
        
        for file_name in os.listdir(subfolder_path):
            
            if file_name.lower().endswith((".bmp")):
                
                image_path = os.path.join(subfolder_path, file_name)
                image = cv2.imread(image_path)

                if image is not None:
                    
                    laplace = LaplacianAnalysis(image)
                    
                    LoGVar = laplace.calculate_variance()
                    LoGQR = laplace.calculate_quantile_range()
                    
                    LoG_Variance.append(LoGVar)
                    LoG_Range.append(LoGQR)
    
    print(f"\n{subfolder} -------------------------------------------------------\n")
    print("Statistics on LoG Variance\n")
    descriptive_statistics(LoG_Variance)

    print("\nStatistics on LoG Range\n")
    descriptive_statistics(LoG_Range)


Cut-marks -------------------------------------------------------

Statistics on LoG Variance

Shapiro results w = 0.71, p = 2.9173964228086313e-30

Min: 11.90
Median: 22.137
NMAD: 7.623
95% Quantile Intevals: [14.07, 70.19] %
Max: 134.51

Statistics on LoG Range

Shapiro results w = 0.74, p = 6.724150279969169e-29

Min: 12.00
Median: 18.000
NMAD: 2.965
95% Quantile Intevals: [16.00, 32.00] %
Max: 48.00

Scores -------------------------------------------------------

Statistics on LoG Variance

Shapiro results w = 0.81, p = 1.0173920793062388e-22

Min: 10.46
Median: 41.893
NMAD: 21.924
95% Quantile Intevals: [13.55, 133.33] %
Max: 241.57

Statistics on LoG Range

Shapiro results w = 0.92, p = 3.761795715017226e-15

Min: 12.00
Median: 24.000
NMAD: 5.930
95% Quantile Intevals: [12.00, 46.00] %
Max: 62.00

Tramplings -------------------------------------------------------

Statistics on LoG Variance

Shapiro results w = 0.54, p = 1.0151336899365777e-19

Min: 9.38
Median: 44.064
NMAD: 30.

In [4]:
main_directory = "./DS1"

for subfolder in os.listdir(main_directory):
    
    percentages = []
    
    subfolder_path = os.path.join(main_directory, subfolder)
    
    if os.path.isdir(subfolder_path):
        
        for file_name in os.listdir(subfolder_path):
            
            if file_name.lower().endswith((".bmp")):
                
                image_path = os.path.join(subfolder_path, file_name)
                image = cv2.imread(image_path)

                if image is not None:
                    
                    canny = CannyEdgeAnalysis(image)
                    
                    percentages.append(canny.calculate_features())
    
    print(f"\n{subfolder} -------------------------------------------------------\n")
    print("Statistics on Percentage of Detectable Features\n")
    descriptive_statistics(percentages, percentage = True)



Croc -------------------------------------------------------

Statistics on Percentage of Detectable Features

Shapiro results w = 0.96, p = 0.16441012918949127

Min: 25.58 %
Mean: 49.896%
Standard Deviation: 9.726
95% Quantile Intevals: [26.75, 65.74] %
Max: 68.54 %

Cut Mark -------------------------------------------------------

Statistics on Percentage of Detectable Features

Shapiro results w = 0.98, p = 4.6825967729091644e-05

Min: 9.68 %
Median: 33.363 %
NMAD: 11.766
95% Quantile Intevals: [16.30, 56.69] %
Max: 61.12 %


In [5]:
main_directory = "./DS3"

for subfolder in os.listdir(main_directory):
    
    percentages = []
    
    subfolder_path = os.path.join(main_directory, subfolder)
    
    if os.path.isdir(subfolder_path):
        
        for file_name in os.listdir(subfolder_path):
            
            if file_name.lower().endswith((".bmp")):
                
                image_path = os.path.join(subfolder_path, file_name)
                image = cv2.imread(image_path)

                if image is not None:
                    
                    canny = CannyEdgeAnalysis(image)
                    
                    percentages.append(canny.calculate_features())
    
    print(f"\n{subfolder} -------------------------------------------------------\n")
    print("Statistics on Percentage of Detectable Features\n")
    descriptive_statistics(percentages, percentage = True)


Cut-marks -------------------------------------------------------

Statistics on Percentage of Detectable Features

Shapiro results w = 0.97, p = 9.007735002342088e-09

Min: 9.68 %
Median: 35.684 %
NMAD: 13.635
95% Quantile Intevals: [16.60, 66.27] %
Max: 74.33 %

Scores -------------------------------------------------------

Statistics on Percentage of Detectable Features

Shapiro results w = 0.99, p = 0.00012168021203251556

Min: 4.80 %
Median: 48.304 %
NMAD: 13.263
95% Quantile Intevals: [14.66, 68.32] %
Max: 79.03 %

Tramplings -------------------------------------------------------

Statistics on Percentage of Detectable Features

Shapiro results w = 0.98, p = 0.03649001941084862

Min: 0.87 %
Mean: 45.183%
Standard Deviation: 20.868
95% Quantile Intevals: [2.71, 87.83] %
Max: 88.56 %


In [6]:
main_directory = "./DS1"

for subfolder in os.listdir(main_directory):
    
    fft_values = []
    
    subfolder_path = os.path.join(main_directory, subfolder)
    
    if os.path.isdir(subfolder_path):
        
        for file_name in os.listdir(subfolder_path):
            
            if file_name.lower().endswith((".bmp")):
                
                image_path = os.path.join(subfolder_path, file_name)
                image = cv2.imread(image_path)

                if image is not None:
                    
                    FFT = FastFourierTransform(image)
                    
                    fft_values.append(FFT.calculate_FFT())
    
    print(f"\n{subfolder} -------------------------------------------------------\n")
    print("Statistics on Magnitude of Representation\n")
    descriptive_statistics(fft_values)


Croc -------------------------------------------------------

Statistics on Magnitude of Representation

Shapiro results w = 0.98, p = 0.682216227054596

Min: 5.34
Mean: 18.988
Standard Deviation: 6.497
95% Quantile Intevals: [5.66, 30.73] %
Max: 32.25

Cut Mark -------------------------------------------------------

Statistics on Magnitude of Representation

Shapiro results w = 0.98, p = 2.749485474851099e-07

Min: -3.32
Median: 9.081
NMAD: 6.296
95% Quantile Intevals: [0.55, 24.05] %
Max: 29.75


In [7]:
main_directory = "./DS3"

for subfolder in os.listdir(main_directory):
    
    fft_values = []
    
    subfolder_path = os.path.join(main_directory, subfolder)
    
    if os.path.isdir(subfolder_path):
        
        for file_name in os.listdir(subfolder_path):
            
            if file_name.lower().endswith((".bmp")):
                
                image_path = os.path.join(subfolder_path, file_name)
                image = cv2.imread(image_path)

                if image is not None:
                    
                    FFT = FastFourierTransform(image)
                    
                    fft_values.append(FFT.calculate_FFT())
    
    print(f"\n{subfolder} -------------------------------------------------------\n")
    print("Statistics on Magnitude of Representation\n")
    descriptive_statistics(fft_values)


Cut-marks -------------------------------------------------------

Statistics on Magnitude of Representation

Shapiro results w = 0.97, p = 1.057175236240937e-08

Min: -3.32
Median: 8.741
NMAD: 5.955
95% Quantile Intevals: [0.65, 22.90] %
Max: 29.75

Scores -------------------------------------------------------

Statistics on Magnitude of Representation

Shapiro results w = 0.98, p = 1.2750085261359345e-05

Min: -7.03
Median: 16.206
NMAD: 6.038
95% Quantile Intevals: [-0.18, 28.03] %
Max: 35.39

Tramplings -------------------------------------------------------

Statistics on Magnitude of Representation

Shapiro results w = 0.68, p = 1.2990340801064286e-16

Min: -12.13
Median: 9.712
NMAD: 10.194
95% Quantile Intevals: [-6.72, 93.19] %
Max: 94.16


In [8]:
main_directory = "./DS1"
sample_total = []
sample_number = []
sample_name = []

for subfolder in os.listdir(main_directory):
    
    sample_name.append(subfolder)
    number_of_good_quality_images = 0
    number_of_images = 0
    
    subfolder_path = os.path.join(main_directory, subfolder)
    
    if os.path.isdir(subfolder_path):
        
        for file_name in os.listdir(subfolder_path):
            
            if file_name.lower().endswith((".bmp")):
                
                image_path = os.path.join(subfolder_path, file_name)
                image = cv2.imread(image_path)

                if image is not None:
                    
                    number_of_images += 1
                    
                    image_contrast = loadImage(image).adequate_contrast
                    
                    if image_contrast:
                        
                        number_of_good_quality_images +=1
    
    sample_total.append(number_of_images)
    sample_number.append(number_of_good_quality_images)
    
for sample in range(len(sample_name)):
    
    target_percentage = (sample_number[sample] / sample_total[sample]) * 100
    
    print(
        f"{target_percentage:.2f} % of {sample_name[sample]}",
        "images are of sufficient contrast for CV applications"
    )

95.65 % of Croc images are of sufficient contrast for CV applications
44.99 % of Cut Mark images are of sufficient contrast for CV applications


In [9]:
main_directory = "./DS3"
sample_total = []
sample_number = []
sample_name = []

for subfolder in os.listdir(main_directory):
    
    sample_name.append(subfolder)
    number_of_good_quality_images = 0
    number_of_images = 0
    
    subfolder_path = os.path.join(main_directory, subfolder)
    
    if os.path.isdir(subfolder_path):
        
        for file_name in os.listdir(subfolder_path):
            
            if file_name.lower().endswith((".bmp")):
                
                image_path = os.path.join(subfolder_path, file_name)
                image = cv2.imread(image_path)

                if image is not None:
                    
                    number_of_images += 1
                    
                    image_contrast = loadImage(image).adequate_contrast
                    
                    if image_contrast:
                        
                        number_of_good_quality_images +=1
    
    sample_total.append(number_of_images)
    sample_number.append(number_of_good_quality_images)
    
for sample in range(len(sample_name)):
    
    target_percentage = (sample_number[sample] / sample_total[sample]) * 100
    
    print(
        f"{target_percentage:.2f} % of {sample_name[sample]}",
        "images are of sufficient contrast for CV applications"
    )

39.62 % of Cut-marks images are of sufficient contrast for CV applications
80.92 % of Scores images are of sufficient contrast for CV applications
47.33 % of Tramplings images are of sufficient contrast for CV applications


In [10]:
main_directory = "./DS1"

for subfolder in os.listdir(main_directory):
    
    number_of_images = 0
    clean_detected_spectral_reflection = 0
    percentage_pixels_clean = []
    
    subfolder_path = os.path.join(main_directory, subfolder)
    
    if os.path.isdir(subfolder_path):
        
        for file_name in os.listdir(subfolder_path):
            
            if file_name.lower().endswith((".bmp")):
                
                image_path = os.path.join(subfolder_path, file_name)
                image = cv2.imread(image_path)

                if image is not None:
                    
                    number_of_images += 1
                    
                    image_spec = SpectralReflection(image)
                    
                    if (image_spec.detected_clean):
                        clean_detected_spectral_reflection += 1
                        percentage_pixels_clean.append(image_spec.percentage_clean)
    
    print(f"\n{subfolder} -------------------------------------------------------\n")
    
    target_percentage = (clean_detected_spectral_reflection / number_of_images) * 100
    
    print(f"{target_percentage:.2f} % of {subfolder} images present cases of specular reflectance")
    
    print("\nStatistics on Percentage of Pixels presenting abnormally high intensity values\n")
    descriptive_statistics(percentage_pixels_clean)


Croc -------------------------------------------------------

58.70 % of Croc images present cases of specular reflectance

Statistics on Percentage of Pixels presenting abnormally high intensity values

Shapiro results w = 0.42, p = 3.013176819166574e-09

Min: 0.00
Median: 0.045
NMAD: 0.065
95% Quantile Intevals: [0.00, 2.55] %
Max: 2.55

Cut Mark -------------------------------------------------------

11.25 % of Cut Mark images present cases of specular reflectance

Statistics on Percentage of Pixels presenting abnormally high intensity values

Shapiro results w = 0.24, p = 2.5479852196240785e-15

Min: 0.00
Median: 0.005
NMAD: 0.007
95% Quantile Intevals: [0.00, 0.61] %
Max: 2.62


In [11]:
main_directory = "./DS3"

for subfolder in os.listdir(main_directory):
    
    number_of_images = 0
    clean_detected_spectral_reflection = 0
    percentage_pixels_clean = []
    
    subfolder_path = os.path.join(main_directory, subfolder)
    
    if os.path.isdir(subfolder_path):
        
        for file_name in os.listdir(subfolder_path):
            
            if file_name.lower().endswith((".bmp")):
                
                image_path = os.path.join(subfolder_path, file_name)
                image = cv2.imread(image_path)

                if image is not None:
                    
                    number_of_images += 1
                    
                    image_spec = SpectralReflection(image)
                    
                    if (image_spec.detected_clean):
                        clean_detected_spectral_reflection += 1
                        percentage_pixels_clean.append(image_spec.percentage_clean)
    
    print(f"\n{subfolder} -------------------------------------------------------\n")
    
    target_percentage = (clean_detected_spectral_reflection / number_of_images) * 100
    
    print(f"{target_percentage:.2f} % of {subfolder} images present cases of specular reflectance")
    
    print("\nStatistics on Percentage of Pixels presenting abnormally high intensity values\n")
    descriptive_statistics(percentage_pixels_clean)


Cut-marks -------------------------------------------------------

9.77 % of Cut-marks images present cases of specular reflectance

Statistics on Percentage of Pixels presenting abnormally high intensity values

Shapiro results w = 0.25, p = 2.074311690025265e-15

Min: 0.00
Median: 0.006
NMAD: 0.008
95% Quantile Intevals: [0.00, 0.61] %
Max: 2.62

Scores -------------------------------------------------------

62.06 % of Scores images present cases of specular reflectance

Statistics on Percentage of Pixels presenting abnormally high intensity values

Shapiro results w = 0.32, p = 4.9171398892636615e-31

Min: 0.00
Median: 0.014
NMAD: 0.020
95% Quantile Intevals: [0.00, 0.91] %
Max: 5.32

Tramplings -------------------------------------------------------

40.00 % of Tramplings images present cases of specular reflectance

Statistics on Percentage of Pixels presenting abnormally high intensity values

Shapiro results w = 0.53, p = 1.44901755524951e-12

Min: 0.00
Median: 0.013
NMAD: 0.0